In [119]:
from bs4 import BeautifulSoup
import unidecode
import re
from sec_downloader import Downloader

import requests
from fake_useragent import UserAgent
import unicodedata
import html

In [33]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [27]:
#soup = BeautifulSoup(get_html("AAPL", "10-K"))

C:\Users\310\AppData\Local\Temp\ipykernel_4428\812151145.py:1: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(get_html("AAPL", "10-K"))


In [267]:
url = "https://www.sec.gov/Archives/edgar/data/729986/000119312524053462/d815282d10k.htm"

response = requests.get(url, headers=get_random_headers())
response.raise_for_status()  # Check that the request was successful

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

## Find all the table tags and decompose them to remove from the soup
#for table in soup.find_all('table'):
#    table.decompose()
#
## Now the soup object doesn't contain the tables, you can extract text
#text = soup.get_text(separator=' ', strip=True)

In [268]:
print(soup.title)

<title>Form 10-K</title>


In [269]:
text = soup.get_text()
text = unidecode.unidecode(text)
text = re.sub(r'\s+', ' ', text).strip()

print(len(text), text[:1000])


620733 Form 10-K falseFY--12-31UNITED BANKSHARES INC/WV0000729986 0000729986 2023-12-31 0000729986 2022-12-31 0000729986 2021-01-01 2021-12-31 0000729986 2022-01-01 2022-12-31 0000729986 2023-01-01 2023-12-31 0000729986 2020-05-12 0000729986 2021-12-31 0000729986 2024-01-31 0000729986 2023-06-30 0000729986 2022-10-01 2022-12-31 0000729986 2022-07-01 2022-09-30 0000729986 2022-04-01 2022-06-30 0000729986 2022-01-01 2022-03-31 0000729986 2023-04-01 2023-06-30 0000729986 2023-07-01 2023-09-30 0000729986 2023-10-01 2023-12-31 0000729986 2023-01-01 2023-03-31 0000729986 2020-12-31 0000729986 ubsi:NonownerOccupiedCommercialRealEstateMember 2022-12-31 0000729986 ubsi:OwnerOccupiedCommercialRealEstateMember 2022-12-31 0000729986 ubsi:OtherCommercialLoansAndLeasesMember 2022-12-31 0000729986 us-gaap:CoreDepositsMember us-gaap:MortgageBankingMember 2022-12-31 0000729986 us-gaap:CoreDepositsMember 2022-12-31 0000729986 ubsi:CrescentTradeNameMember us-gaap:MortgageBankingMember 2022-12-31 00007299

In [270]:
pattern_mnda = r"(?i)item\s+\d+[A-Za-z]?\.?\.\s+management[’'`]s\s+Discussion\s+and\s+Analysis\s+of\s+Financial\s+Condition\s+and\s+Results\s+of\s+Operations"

pattern_qnq = r"(?i)item\s+\d+[A-Za-z]?\.?\s+quantitative\s+and\s+qualitative\s+disclosures\s+about\s+market\s+risk"

In [271]:
matches_mnda = re.findall(pattern_mnda, text)
matches_qnq = re.findall(pattern_qnq, text)
print(matches_mnda, '\n', matches_qnq)

["Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", "Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS"] 
 ['Item 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK', 'Item 7A. QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK']


In [283]:
matches_mnda = re.finditer(pattern_mnda, text)
matches_qnq = re.finditer(pattern_qnq, text)

for i, mnda in enumerate(matches_mnda):
    print(f'mnda: {mnda.end()}')
for x, qnq in enumerate(matches_qnq):
    print(f'qnq: {qnq.start()}')
    
    
    #print(f'mnda №{i+1}:{mnda.span()} qnq №{x+1}: {qnq.span()}')

mnda: 99293
mnda: 246642
qnq: 99297
qnq: 346280


In [288]:
import re

def find_longest_substring(text, pattern_mnda, pattern_qnq):
    matches_mnda = list(re.finditer(pattern_mnda, text))
    matches_qnq = list(re.finditer(pattern_qnq, text))
    
    longest_substring = ""
    
    for mnda_match in matches_mnda:
        mnda_end = mnda_match.end()
        # Find the first matches_qnq after this mnda_match
        for qnq_match in matches_qnq:
            if qnq_match.start() > mnda_end:
                substring = text[mnda_end:qnq_match.start()]
                if len(substring) > len(longest_substring):
                    longest_substring = substring
                break  # We only want the first qnq match after each mnda match
    
    return longest_substring

# Example usage
longest = find_longest_substring(text, pattern_mnda, pattern_qnq)
print(f"The longest substring found is: '{len(longest)}'")


The longest substring found is: '99638'
